<a href="https://colab.research.google.com/github/ndb796/Management-Programs-for-YouTuber/blob/main/Comment_Management_Template_(YouTube_Data_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### <b>필요한 라이브러리 불러오기</b>

In [1]:
from google.colab import files
import google_auth_oauthlib.flow
import googleapiclient.discovery

#### <b>Client Secret 파일 불러오기</b>

In [2]:
uploaded = files.upload()

Saving client_secret.json to client_secret.json


#### <b>API를 사용하기 위해 구글 계정으로 인증하기</b>

In [ ]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
CLIENT_SECRET_FILE = 'client_secret.json'

# [ 사용할 권한 명시 ]
# 1. 유튜브 계정 보기(youtube.readonly)
# 2. 유튜브 동영상, 평가, 댓글, 자막 보기, 수정 및 완전 삭제(youtube.force-ssl)
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly', 'https://www.googleapis.com/auth/youtube.force-ssl']

# API 클라이언트 생성 및 인증하여 Credential 객체 얻기
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

#### <b>해당 사용자의 채널 정보 불러오기</b>

In [14]:
# 인증된 사용자 채널의 기본 정보 가져오기
channels_response = youtube.channels().list(
    mine=True,
    part='contentDetails'
).execute()

channel = channels_response['items'][0] # 첫 번째 채널 선택
uploads_playlist_id = channel['contentDetails']['relatedPlaylists']['uploads'] # 업로드 영상 플레이 리스트 ID 추출

print('업로드한 영상의 Playlist ID:', uploads_playlist_id)

업로드한 영상의 Playlist ID: UUhflhu32f5EUHlY7_SetNWw


#### <b>해당 사용자의 동영상 ID 불러오기</b>

In [15]:
# 인증된 사용자 채널의 모든 비디오 정보 가져오기
playlistitems_list_request = youtube.playlistItems().list(
    playlistId=uploads_playlist_id,
    part='snippet',
    maxResults=50
)

cnt = 0
last = 20 # 최근 몇 개까지의 동영상을 확인할지 설정
video_list = []

# 해당 플레이 리스트의 모든 동영상을 하나씩 확인하며
while playlistitems_list_request:
    playlistitems_list_response = playlistitems_list_request.execute()

    # 각 비디오(video)에 대한 정보 출력
    for playlist_item in playlistitems_list_response['items']:
        video_id = playlist_item['snippet']['resourceId']['videoId']
        title = playlist_item['snippet']['title']
        video_list.append((video_id, title))
        cnt += 1
        if cnt >= last:
            break

    if cnt >= last:
        break
    playlistitems_list_request = youtube.playlistItems().list_next(playlistitems_list_request, playlistitems_list_response)

print(f'{cnt}개의 동영상 정보를 불러왔습니다.')

20개의 동영상 정보를 불러왔습니다.


In [16]:
for (video_id, title) in video_list:
    print(f'Video ID: {video_id} / 제목: {title}')

Video ID: va3e2c4uKJk / 제목: CNN을 활용한 스타일 전송(Style Transfer) | 꼼꼼한 딥러닝 논문 리뷰와 코드 실습
Video ID: OM-6zYYRYfg / 제목: Arbitrary Style Transfer in Real-time with Adaptive Instance Normalization (꼼꼼한 딥러닝 논문 리뷰와 코드 실습)
Video ID: HXgfw3Z5zRo / 제목: StyleGAN: 고해상도 가상 얼굴 이미지 생성 기술 (꼼꼼한 딥러닝 논문 리뷰와 코드 실습)
Video ID: KqqlQKKitLM / 제목: [같이 불꽃 코딩해요] 7시간 논스톱 코딩 타임랩스 (7 Hours Non-stop Coding Time-lapse)
Video ID: 58fuWVu5DVU / 제목: 배치 정규화(Batch Normalization) [꼼꼼한 딥러닝 논문 리뷰와 코드 실습]
Video ID: 28XjFzn0ktA / 제목: [썰방송] 문과 내신 6등급(과천외고)이었는데 성적 맞춰 꼴찌로 들어간 컴퓨터 학과에서 5시간 자며 빡공해서 수석 졸업하고 스펙 괴물이 되어서 24살 때 연수익 1억을 만들었던 썰
Video ID: AVvlDmhHgC4 / 제목: GAN: Generative Adversarial Networks (꼼꼼한 딥러닝 논문 리뷰와 코드 실습)
Video ID: AA621UofTUA / 제목: [딥러닝 기계 번역] Transformer: Attention Is All You Need (꼼꼼한 딥러닝 논문 리뷰와 코드 실습)
Video ID: EF-IYFTKZiE / 제목: [텍스트 분류 모델 공격 기법] TextFooler: Is BERT Really Robust?
Video ID: 4DzKM0vgG1Y / 제목: [딥러닝 기계 번역] Seq2Seq: Sequence to Sequence Learning with Neural Networks (꼼꼼한 딥러닝 논문 리뷰와 코드 실습)
Video ID: i5y

#### <b>댓글 스레드(Comment Thread) 불러오기</b>

In [17]:
# commentThreads.list() API를 호출하여 특정 비디오의 댓글 스레드(thread)를 불러오기
def get_comment_threads(youtube, video_id):
    results = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        textFormat='plainText',
        maxResults=100, # 최근 100개까지의 댓글 확인
    ).execute()

    comment_list = []
    for item in results['items']:
        comment_id = item['id']
        comment = item['snippet']['topLevelComment']
        author = comment['snippet']['authorDisplayName']
        publishedAt = comment['snippet']['publishedAt']
        text = comment['snippet']['textDisplay']
        comment_list.append((comment_id, author, publishedAt, text))

    return comment_list

In [18]:
video_id = 'ukkLCl9yBvE' # 비디오 ID
comment_list = get_comment_threads(youtube, video_id)

for (comment_id, author, publishedAt, text) in comment_list:
    print(f'[ Comment ID: {comment_id} / 작성자: {author} / 작성 날짜: {publishedAt} ]')
    print(text)

[ Comment ID: Ugzs9DEv501r_8nG8RZ4AaABAg / 작성자: 연습 / 작성 날짜: 2021-02-05T13:56:36Z ]
마지막에 심멎할뻔 했네요.....(저 남잔데)
[ Comment ID: Ugxg-fepvSLBiMe3Ob14AaABAg / 작성자: 김정윤 / 작성 날짜: 2021-02-05T12:49:44Z ]
잘 봤습니다.
[ Comment ID: UgyGJJa5uwNSJVR3zb54AaABAg / 작성자: Kevin Lim / 작성 날짜: 2021-02-05T11:27:05Z ]
제 월급을 바칠테니 코딩 좀 알려주실래요?
[ Comment ID: UgxGkWrhviQuAPF1UMR4AaABAg / 작성자: 호망두 / 작성 날짜: 2021-02-05T11:16:17Z ]
나이가 많아도 괜찮을 까요?
[ Comment ID: UgxtC3cWigqAQZGrmDx4AaABAg / 작성자: K도로도롱 / 작성 날짜: 2021-02-04T22:44:56Z ]
너무너무너무 원했던 자료입니다 감사합니다
[ Comment ID: UgxtSS_QsLLUtgmG1-J4AaABAg / 작성자: JuNo kim / 작성 날짜: 2021-01-31T13:16:40Z ]
기본 알고리즘 공부를 하고 그리디,완전탐색을 푸시라는게 기본적으로 정렬,DFS/BFS,재귀,백트래킹 이런거는 일단 뭔지는 알고 그다음에 그리디랑 완전탐색으로 알고리즘으로 넘어가라는 말씀이시죠? 그냥 건들지도 말고 바로 그리디랑 완전탐색 풀라는거 아니죠?(기초100제 풀고 그냥 바로 그리디로 갔더니 정렬이랑 이런게 많이 나와서 ㄷㄷ)
[ Comment ID: UgzC4O64SBp1UMD9us14AaABAg / 작성자: laskz a / 작성 날짜: 2021-01-31T08:18:00Z ]
코딩테스트  시험볼때 문제마다 유동적으로 c++,  python 이렇게 풀수있나요?
예를들면 1번은 c++ , 2번은 python
[ Comment ID: UgxG4fgLVdX1uRe4y1B4AaABA

#### <b>댓글 스레드 삭제 및 삭제된 스레드 복구 방법</b>

In [19]:
comment_ids = ['Ugzk1Ct6DqtrHjxHl5N4AaABAg', 'UgyEeP3bVrIH6smvlsl4AaABAg'] # 댓글 ID 설정
banned = True # 댓글 작성자 차단 여부

# 삭제 수행 (삭제 이후에는 더 이상 API 상에서도 해당 댓글이 보이지 않음)
request = youtube.comments().setModerationStatus(
    id=comment_ids,
    moderationStatus='rejected',
    banAuthor=banned
)
request.execute()

print(f'{len(comment_ids)}개의 댓글을 삭제했습니다.')

2개의 댓글을 삭제했습니다.


In [20]:
comment_ids = ['Ugzk1Ct6DqtrHjxHl5N4AaABAg', 'UgyEeP3bVrIH6smvlsl4AaABAg'] # 댓글 ID 설정
banned = False # 댓글 작성자 차단 여부

# 다시 보이게 만들기 (댓글 ID를 기억하고 있는 경우에는 다시 보이게 만들 수 있음)
request = youtube.comments().setModerationStatus(
    id=comment_ids,
    moderationStatus='published',
    banAuthor=banned
)
request.execute()

print(f'{len(comment_ids)}개의 댓글을 복구했습니다.')

2개의 댓글을 복구했습니다.
